--------

***Keras Callbacks and Tensorboard***

------

So until now we used set the model parameters and trained the model. Once the mode.fit() was called we didn't have any control on the model training. Most of the cases we ended up overfitting and then had to retrain. This is very impractical. Just to avoid this Keras callbacks can be used. The Tensorboard is a visualization tool where we can log the data and visualize all the parameters of the model as the model is getting trained. This can again help understand when we go wrong and abort the training at the right time. Also visualizing the weights histograms can tell a lot about how the model parameters distributions are. 



-----

***Callbacks***

-----

When should we stop training the model ? 
The simple answer is when we see that the validation loss is not decreasing anymore. This is the point when the model is trained and starts overfitting to the training data. THe training error is always going to decrease as the optimization function we have written is meant to do so. 

We can achieve this by using Keras callbacks. A ***callback*** is an object that is passed to the model.fit(). So, when the model is training it will call this function during training. It has access to all the data, state of the model, performance, and it can take action, i.e. stop training, save model, load different weights set, or alter the state of the model. 

Some ways in which the callback ca be used are - 

- **Model Checkpointing** - Saving the current weights at different checkpoints during training. 
- **Early stopping** - Interrupting training when the validation loss is no more improving. 
- **Dynamically adjusting model parameters** - Adjusting the weights of the optimizer. 
- **Logging training and validation metrics during training, or visualizing the representation learned by the model as they've updated** 

------

***Keras Callbacks***

-----

The keras callbacks include a number of built in callbacks. example keras.callbacks.ModelCheckpoint, keras.callbacks.EarlyStopping, etc. 

Let's dive in and understand them a little better on how they can they be used. 

```python
import keras

callbacks_list  = [keras.callbacks.EarlyStopping(monitor = 'acc', patience = 1,),
                  keras.callbacks.ModelCheckpoint(filePath = 'my_model.h5', monitor = 'val_loss', save_best_only = True,)]

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])

model.fit(x, y, epochs = 100, batch_size = 32, 
         callbacks = callbacks_list, validation_data = (x_val, y_val))
```
--------

***ReduceLROnPlateau Callback***

-------

```python
callbacks_list  = [keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 10,)]
```

The function reduces the learning rate by a factor of 10 as soon as the modelvalidation loss has stopped improving. 

------
***Writing your own callback***

-------
If a specific action needs to be taken which is not covered in Keras callbacks, this function can be used. 
Callbacks are implemented by sub-classing the class *keras.callbacks.Callback*. 

It can be implemented in a number of ways, by calling at various points during the training like 
```python
on_epoch_begin
on_epoch_end

on_batch_begin
on_batch_end

on_train_begin
on_train_end
```

These methods are called using the logs argument, which is a dictionary containing information about previous batch, or epoch, or training run: training and validation metrics, and so on. 

The callbacks also has access to following attributes. 

```python
self.model ## Instance from which the model is called
self.validation_data ## The value of what was passed to fit as validation data
```

**Here is an example of custom callback that saves to disk the activations of every layer of model at the end of every epoch, calculated on first sample of validation data**

```python
import keras
import numpy as np 

class ActivationLogger(keras.callbacks.Callback):
    
    ### Called by parent model before training, to inform the callback of what model will be calling it.
    def set_model(self, model):
        self.model - model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.modelsModel(model.input, layer_outputs)
    
    def on_epoch_end(self, epoch, logs = None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation data')
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_' + str(epoch) + '.npz', 'w')
        np.savez(f, activations)
        f.close()
  ```
  

--------------------

***Tensorboard***

----------------

Tensorboard is a browser based visualization tool, where eseentially everything about the model can be visualized, the model graph, weights distribution, learning_rate graph, learning graphs, visualizing activations and gradients etc

Let's start by using tensorboard. 

The tensorboard is hosted on localhost at port 6006. 

In [1]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
max_features = 2000
max_len = 500 

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)
x_train = sequence.pad_sequences(x_train, maxlen = max_len)
x_test = sequence.pad_sequences(x_test, maxlen = max_len)

In [4]:
model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length = max_len, name = 'embed'))
model.add(layers.Conv1D(32,7,activation = 'relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32,7,activation = 'relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937.0
Trainable params: 291,937.0
Non-trainable params: 0.0
________________________________________________________________

In [6]:
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


***Create a directory in the same folder where the tensorboard logs will be saved***

In [7]:
callbacks = [keras.callbacks.TensorBoard(log_dir = 'my_log_dir')]

In [8]:
history = model.fit(x_train, y_train, epochs = 20, batch_size = 128, validation_split = 0.2, callbacks = callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 3s - loss: 0.6948 - acc: 0.5935 - val_loss: 0.6780 - val_acc: 0.7030
Epoch 2/20
20000/20000 [==============================] - 2s - loss: 0.5310 - acc: 0.8036 - val_loss: 0.4340 - val_acc: 0.8300
Epoch 3/20
20000/20000 [==============================] - 2s - loss: 0.4869 - acc: 0.8100 - val_loss: 0.4898 - val_acc: 0.8132
Epoch 4/20
20000/20000 [==============================] - 2s - loss: 0.4446 - acc: 0.7963 - val_loss: 1.3151 - val_acc: 0.5738
Epoch 5/20
20000/20000 [==============================] - 2s - loss: 0.4506 - acc: 0.7829 - val_loss: 0.5196 - val_acc: 0.7752
Epoch 6/20
20000/20000 [==============================] - 2s - loss: 0.4238 - acc: 0.7676 - val_loss: 0.5990 - val_acc: 0.7256
Epoch 7/20
20000/20000 [==============================] - 2s - loss: 0.4022 - acc: 0.7600 - val_loss: 0.5574 - val_acc: 0.7300
Epoch 8/20
20000/20000 [==============================] - 2s -

***The Tensorboard can be passed parameters like histograms, embedding_freq etc, for more analysis.***

***Keras also allows to save model graphs as images***

In [9]:
from keras.utils import plot_model

In [11]:
plot_model(model, show_shapes = True, to_file = 'model.png')